In [ ]:
# Returns XOR of 'a' and 'b' 
def xor(a, b): 
    result = [] 
    for i in range(1, len(b)): 
        if a[i] == b[i]: 
            result.append('0') 
        else: 
            result.append('1')   
    return ''.join(result)    # String conversion

# Performs Modulo-2 division 
def mod2div(divident, divisor): 
    
    # Number of bits to be XORed at a time. 
    pick = len(divisor) 
  
    # Slicing the divident to appropriate 
    # length for particular step 
    tmp = divident[0 : pick] 
  
    while pick < len(divident): 
  
        if tmp[0] == '1': 
  
            # replace the divident by the result 
            # of XOR and pull 1 bit down 
            tmp = xor(divisor, tmp) + divident[pick] 
  
        else:   # If leftmost bit is '0' 
            # If the leftmost bit of the dividend (or the 
            # part used in each step) is 0, the step cannot 
            # use the regular divisor; we need to use an 
            # all-0s divisor. 
            tmp = xor('0'*pick, tmp) + divident[pick] 
  
        # increment pick to move further 
        pick += 1
  
    # For the last n bits, we have to carry it out 
    # normally as increased value of pick will cause 
    # Index Out of Bounds. 
    if tmp[0] == '1': 
        tmp = xor(divisor, tmp) 
    else: 
        tmp = xor('0'*pick, tmp) 
  
    checkword = tmp 
    return checkword 
  
# Function used at the sender side to encode 
# data by appending remainder of modular divison 
# at the end of data. 

def encodeData(data, key): 
  
    remainder = mod2div(data, key) 
    print("Remainder : ", remainder)
    return remainder

In [ ]:
import socket             
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
#AF_INET= IPv4, SOCK_STREM=TCP
print("Socket successfully created")

port = 8000               
s.bind(('', port))         
print ("socket binded to %s" %(port)) 

s.listen(10)      
print ("socket is listening")            
key="1001"

count=0
while True: 
    c, addr = s.accept()
    count=count+1
    print ('Got connection from', addr)
    f = open("output.txt", "w")
    message='Thank you for connecting, Client no. %s' %(count)
    c.send(message.encode('utf-8'))

    while True:
        recieve= c.recv(1024).decode('utf-8')
        if recieve=="EoF":
            f.close()
            break
        
        r=recieve[0:-3]                  # Without remainder bit i.e., actual string
        remainder=encodeData(recieve,key)
        if int(remainder) == 0:
            j=0
            k=7
            for i in range(int(len(r)/7)):
                n=r[j:k]
                j+=7
                k+=7
                f.write(chr(int(n,2)))
                
            c.send("OK".encode('utf-8'))
            
        else:
            c.send("NAK".encode('utf-8'))    